In [1]:
from llama_index.core.tools import FunctionTool
from tools import add, multiply
from agent import Agent
from chat import ChatWithAgent
from handoff import SequentialHandoffAgent

In [2]:
def add_wechat_account(account:str):
    """Add the customer's wechat account to the system. before adding, you should confirm the account format is correct."""
    print(f"Add wechat account: {account}")
    return f"Wechat account {account} added."

customer_service_agent = Agent(system_prompt="""
You are a helpful customer service agent, you answer customer questions and help them solve product issues on their IPHONE devices.
If the customer wants to add your wechat account, use the tool "handoff_to_wechat_customer_service_agent" to handoff to wechat customer service agent.
Be sure to use the tool when answering 微信 or 微信号 related questions."
""",tools=[])

wechat_agent = Agent(system_prompt="""
You are a WeChat customer service agent, you help customers add your wechat account and answer their questions.
You can use the tool "add_wechat_account" to add the customer's wechat account to the system.
""",tools=[
    FunctionTool.from_defaults(
        fn = add_wechat_account,
        name = "add_wechat_account",
        description="Add the customer's wechat account to the system. before adding, you should confirm the account format is correct.",

    )]
)

agent = SequentialHandoffAgent(
    root_agent=customer_service_agent,
    handoff_to_agent=wechat_agent
)


In [3]:
chatter = ChatWithAgent(agent)

In [4]:
await chatter.chat("你好")

=== Chat Messages ===
--user: 你好
你好！有什么可以帮你的吗？

In [5]:
await chatter.chat("怎么用airdrop传文件？用30个字简单告诉我")

=== Chat Messages ===
--user: 你好
--assistant: 你好！有什么可以帮你的吗？
--user: 怎么用airdrop传文件？用30个字简单告诉我
打开Airdrop，选择文件，点击接收设备即可传输。

In [6]:
await chatter.chat("咱们加微信说吧")

=== Chat Messages ===
--user: 你好
--assistant: 你好！有什么可以帮你的吗？
--user: 怎么用airdrop传文件？用30个字简单告诉我
--assistant: 打开Airdrop，选择文件，点击接收设备即可传输。
--user: 咱们加微信说吧
Handoff to WeChat customer service agent


In [7]:
await chatter.chat("微信好113445")

=== Chat Messages ===
--user: 你好
--assistant: 你好！有什么可以帮你的吗？
--user: 怎么用airdrop传文件？用30个字简单告诉我
--assistant: 打开Airdrop，选择文件，点击接收设备即可传输。
--user: 咱们加微信说吧
--assistant: 
--tool: 
--assistant: 
--user: 微信好113445
Add wechat account: 113445
好的，已为您添加微信账号113445。有什么需要帮助的吗？

In [8]:
chatter.messages

[{'role': 'user', 'content': '你好'},
 {'role': 'assistant', 'content': '你好！有什么可以帮你的吗？'},
 {'role': 'user', 'content': '怎么用airdrop传文件？用30个字简单告诉我'},
 {'role': 'assistant', 'content': '打开Airdrop，选择文件，点击接收设备即可传输。'},
 {'role': 'user', 'content': '咱们加微信说吧'},
 {'role': <MessageRole.ASSISTANT: 'assistant'>,
  'additional_kwargs': {'tool_calls': [{'function': {'name': 'handoff_to_wechat_customer_service_agent',
      'arguments': {}}}],
   'thinking': ''},
  'blocks': [{'block_type': 'text', 'text': ''}]},
 {'role': <MessageRole.TOOL: 'tool'>,
  'additional_kwargs': {'tool_call_id': 'handoff_to_wechat_customer_service_agent'},
  'blocks': [{'block_type': 'text',
    'text': 'You are now connected to a WeChat customer service agent.'}]},
 {'role': 'assistant', 'content': ''},
 {'role': 'user', 'content': '微信好113445'},
 {'role': <MessageRole.ASSISTANT: 'assistant'>,
  'additional_kwargs': {'tool_calls': [{'function': {'name': 'add_wechat_account',
      'arguments': {'account': '113445'}}}],
   't